In [2]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [3]:
# Get images
image = img_to_array(load_img('../Alpha-version/woman.jpg'))
image = np.array(image, dtype=float)

In [4]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [5]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

2023-12-11 20:05:04.132880: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-12-11 20:05:04.132901: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-12-11 20:05:04.132907: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-12-11 20:05:04.132938: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-11 20:05:04.132956: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# Finish model
model.compile(optimizer='rmsprop',
			loss='mse')

In [7]:
model.fit(x=X, 
	y=Y,
	batch_size=1,
	epochs=1000)

Epoch 1/1000


2023-12-11 20:05:04.818924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step - loss: 0.0372
Epoch 2/1000
1/1 [==============================] - 0s 57ms/step - loss: 0.0885
Epoch 3/1000
1/1 [==============================] - 0s 46ms/step - loss: 0.0156
Epoch 4/1000
1/1 [==============================] - 0s 48ms/step - loss: 0.0106
Epoch 5/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.0082
Epoch 6/1000
1/1 [==============================] - 0s 53ms/step - loss: 0.0065
Epoch 7/1000
1/1 [==============================] - 0s 47ms/step - loss: 0.0062
Epoch 8/1000
1/1 [==============================] - 0s 50ms/step - loss: 0.0062
Epoch 9/1000
1/1 [==============================] - 0s 48ms/step - loss: 0.0076
Epoch 10/1000
1/1 [==============================] - 0s 49ms/step - loss: 0.0053
Epoch 11/1000
1/1 [==============================] - 0s 50ms/step - loss: 0.0047
Epoch 12/1000
1/1 [==============================] - 0s 47ms/step - loss: 0.0047
Epoch 13/1000
1/1 [==============================] -

In [15]:
print(model.evaluate(X, Y, batch_size=1))
image = img_to_array(load_img('../Alpha-version/swim.jpg'))
image = np.array(image, dtype=float)
X = rgb2lab(1.0/255*image)[:,:,0]
X = X.reshape(1, 400, 400, 1)

output = model.predict(X)
output *= 128
# Output colorizations

from skimage import color, io
import numpy as np


# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]

# Ensure values are within the valid range 0-255 and convert to RGB
cur = np.clip(cur, 0, 255)
cur = color.lab2rgb(cur)

# Convert to uint8 format for the colored image
cur_uint8 = (cur * 255).astype(np.uint8)

# Convert to grayscale and then to uint8 format for the grayscale image
gray_image = color.rgb2gray(cur)
gray_image_uint8 = (gray_image * 255).astype(np.uint8)

# Save the images
io.imsave("img_result.png", cur_uint8)
io.imsave("img_gray_version.png", gray_image_uint8)

1/1 [==============================] - 0s 37ms/step - loss: 0.0060
0.0059678759425878525
1/1 [==============================] - 0s 18ms/step
